In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
# Standard library imports.
import pickle
from collections import Counter

# Related third party imports.
import pandas as pd
import numpy as np

# Local application/library specific imports.
from src.models.network_simulation import NetworkSimulation

# 1. Hypervisor Frequency in the Quartets

In [4]:
network_name = '25_italy'
network_operator_path = f'../data/processed/network_operators/{network_name}/{network_name}_L{5}_k16.p'
network_operator = pickle.load(open(network_operator_path, "rb"))

In [18]:
counts = Counter(q[1] for q in network_operator.quartets) + Counter(q[2] for q in network_operator.quartets)
counts.keys()

dict_keys([0, 7, 18, 6, 8, 1, 12, 17, 11, 15, 20, 10, 16, 9, 19, 2, 22, 23, 14, 3, 5, 24, 13, 21, 4])

In [19]:
network_name = '25_italy'
df = pd.DataFrame()

for L in range(1,11):
    network_operator_path = f'../data/processed/network_operators/{network_name}/{network_name}_L{L}_k16.p'
    network_operator = pickle.load(open(network_operator_path, "rb"))

    counts = Counter(q[1] for q in network_operator.quartets) + Counter(q[2] for q in network_operator.quartets)
    df[L] = pd.Series(counts, index=counts.keys())

In [23]:
df = df.sort_index()
df['mean'] = df.divide(df.sum(), axis='columns').mean(axis=1)

In [24]:
print(df.to_csv())

,1,2,3,4,5,6,7,8,9,10,mean
0,6,28,95,292,917,1919,3617,5357,7200,8899,0.05622873946783301
1,4,11,51,171,651,1541,3098,4914,6761,8537,0.04146340272536753
2,2,6,19,63,243,640,1536,2890,4681,6862,0.022186905308252937
3,4,18,34,69,135,326,777,1501,2578,4376,0.019725752809952184
4,4,18,34,51,103,279,631,1199,2088,3520,0.017587992263637935
5,4,18,34,51,81,260,585,1091,1903,3162,0.016831516671563344
6,8,35,156,523,1299,2328,3620,5205,6790,8404,0.06969007088023205
7,6,33,167,557,1328,2354,3639,5290,6831,8564,0.0694843564858493
8,10,29,222,573,1359,2362,3605,5221,6805,8358,0.07499124033355
9,8,19,104,442,1108,2033,3230,4625,6303,7847,0.057502802067207384
10,4,12,21,164,569,1271,2439,3732,5119,6592,0.03356028403484236
11,2,6,15,42,205,649,1448,2516,3748,5069,0.01901218555071078
12,6,12,101,442,1128,2069,3147,4350,5727,7358,0.05329363739766692
13,4,6,13,57,221,718,1654,2725,4010,4984,0.02170155276978606
14,4,6,20,92,347,1076,2071,3302,4384,5605,0.026632661210433624
15,4,19,133,508,1278,2177,3330,

# 2. Hypervisor Frequency in the Placements

In [66]:
network = '25_italy'
simulation_date = '2021-11-09'
placement_type = 'ilp-opt'

df = pd.read_json(f"../results/{network}/{simulation_date}-{network}-{placement_type}.json")
df.latency_factor = df.latency_factor.round(1)

In [67]:
dfh = pd.DataFrame()
for L, row in df[['latency_factor','h_list']].groupby('latency_factor').sum().iterrows():
    counts = Counter(row['h_list'])
    dfh[L] = pd.Series(counts, index=counts.keys())
dfh = dfh.fillna(0).astype('int')
dfh = dfh.sort_index()
dfh['mean'] = dfh.divide(dfh.sum(), axis='columns').mean(axis=1)

In [ ]:
dfh

In [69]:
network = '25_italy'
simulation_date = '2021-11-09'
hypervisor_frequency = pd.DataFrame()

for placement_type in ['heu-hco', 'ilp-hco', 'ilp-acc', 'ilp-opt']:
    df = pd.read_json(f"../results/{network}/{simulation_date}-{network}-{placement_type}.json")
    df.latency_factor = df.latency_factor.round(1)
    
    dfh = pd.DataFrame()
    for L, row in df[['latency_factor','h_list']].groupby('latency_factor').sum().iterrows():
        counts = Counter(row['h_list'])
        dfh[L] = pd.Series(counts, index=counts.keys())
    dfh = dfh.fillna(0).astype('int')
    dfh = dfh.sort_index()
    hypervisor_frequency[placement_type] = dfh.divide(dfh.sum(), axis='columns').mean(axis=1)

In [80]:
print(hypervisor_frequency.to_csv(float_format='%.3f'))

,heu-hco,ilp-hco,ilp-acc,ilp-opt
0,0.088,0.118,0.072,0.083
1,0.017,0.004,0.029,0.028
2,0.041,0.018,0.031,0.035
3,0.032,0.051,0.044,0.034
4,0.022,0.024,0.013,0.013
5,0.020,0.004,0.024,0.023
6,0.031,0.095,0.030,0.032
7,0.036,0.004,0.030,0.031
8,0.030,0.018,0.038,0.038
9,0.020,0.010,0.016,0.019
10,0.013,0.010,0.010,0.013
11,0.032,0.018,0.018,0.021
12,0.027,0.025,0.024,0.018
13,0.035,0.018,0.025,0.027
14,0.018,0.035,0.019,0.015
15,0.025,0.023,0.030,0.035
16,0.013,0.004,0.002,0.003
17,0.009,0.010,0.009,0.011
18,0.030,0.031,0.026,0.025
19,0.043,0.031,0.035,0.034
20,0.086,0.076,0.128,0.097
21,0.029,0.060,0.019,0.022
22,0.030,,0.007,0.023
23,0.098,0.101,0.098,0.098
24,0.179,0.210,0.223,0.221

